### Deliverable 1: Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],1)
application_df.head()

C:\Users\isler\AppData\Local\Temp/ipykernel_40296/4242059896.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(['EIN','NAME'],1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_count[application_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Look at INCOME_AMT value counts for binning
income_count = application_df.INCOME_AMT.value_counts()
income_count

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [9]:
# Determine which values to replace if counts are less than ..?
replace_income = list(income_count[income_count < 500].index)

# Replace in dataframe
for cls in replace_income:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Other              564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [10]:
# Look at AFFILIATION value counts for binning
affiliate_count = application_df.AFFILIATION.value_counts()
affiliate_count

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [11]:
# Determine which values to replace if counts are less than ..?
replace_affiliate = list(affiliate_count[affiliate_count < 100].index)

# Replace in dataframe
for cls in replace_affiliate:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [12]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               3
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                7
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [13]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

C:\Users\isler\AppData\Local\Temp/ipykernel_40296/108820884.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(application_cat,1)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL','SPECIAL_CONSIDERATIONS_Y'],1).values

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, stratify=y)

C:\Users\isler\AppData\Local\Temp/ipykernel_40296/849330475.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = application_df.drop(['IS_SUCCESSFUL','SPECIAL_CONSIDERATIONS_Y'],1).values


In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 125
hidden_nodes_layer2 = 50


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

# Output layer
nn.add(Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 125)               4750      
                                                                 
 dense_1 (Dense)             (None, 50)                6300      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11,101
Trainable params: 11,101
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 918us/step - loss: 0.5674 - accuracy: 0.7245
Epoch 2/100
804/804 [==============================] - 1s 897us/step - loss: 0.5555 - accuracy: 0.7301
Epoch 3/100
804/804 [==============================] - 1s 894us/step - loss: 0.5522 - accuracy: 0.7317
Epoch 4/100
804/804 [==============================] - 1s 905us/step - loss: 0.5503 - accuracy: 0.7335
Epoch 5/100
804/804 [==============================] - 1s 890us/step - loss: 0.5492 - accuracy: 0.7338
Epoch 6/100
804/804 [==============================] - 1s 907us/step - loss: 0.5478 - accuracy: 0.7344
Epoch 7/100
804/804 [==============================] - 1s 906us/step - loss: 0.5472 - accuracy: 0.7352
Epoch 8/100
804/804 [==============================] - 1s 925us/step - loss: 0.5469 - accuracy: 0.7334
Epoch 9/100
804/804 [==============================] - 1s 911us/step - loss: 0.5460 - accuracy: 0.7355
Epoch 10/100
804/804 [==============================] - 1s 925us/step - l

804/804 [==============================] - 1s 1ms/step - loss: 0.5348 - accuracy: 0.7405
Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5349 - accuracy: 0.7413
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7409
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5345 - accuracy: 0.7407
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5343 - accuracy: 0.7416
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5344 - accuracy: 0.7409
Epoch 86/100
804/804 [==============================] - 1s 918us/step - loss: 0.5338 - accuracy: 0.7414
Epoch 87/100
804/804 [==============================] - 1s 921us/step - loss: 0.5344 - accuracy: 0.7420
Epoch 88/100
804/804 [==============================] - 1s 914us/step - loss: 0.5343 - accuracy: 0.7413
Epoch 89/100
804/804 [==============================] - 1s 934us/step - loss: 0.5340 - ac

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5697 - accuracy: 0.7249 - 270ms/epoch - 1ms/step
Loss: 0.5696961879730225, Accuracy: 0.7248979806900024


In [21]:
# Optimize the Model
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Optimization3/',exist_ok=True)
checkpoint_path = 'checkpoints/Optimization3/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 99/804 [==>...........................] - ETA: 0s - loss: 0.5492 - accuracy: 0.7273
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
177/804 [=====>........................] - ETA: 0s - loss: 0.5389 - accuracy: 0.7396
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
288/804 [=========>....................] - ETA: 0s - loss: 0.5380 - accuracy: 0.7391
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
370/804 [============>.................] - ETA: 0s - loss: 0.5355 - accuracy: 0.7402
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
454/804 [===============>..............] - ETA: 0s - loss: 0.5346 - accuracy: 0.7407
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
545/804 [===================>..........] - ETA: 0s - loss: 0.5325 - accuracy: 0.7414
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
650/804 [=======================>......] - ETA: 0s - loss: 0.5337 - accuracy

223/804 [=======>......................] - ETA: 0s - loss: 0.5376 - accuracy: 0.7363
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
330/804 [===========>..................] - ETA: 0s - loss: 0.5374 - accuracy: 0.7378
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
425/804 [==============>...............] - ETA: 0s - loss: 0.5359 - accuracy: 0.7393
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
526/804 [==================>...........] - ETA: 0s - loss: 0.5376 - accuracy: 0.7385
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
625/804 [======================>.......] - ETA: 0s - loss: 0.5342 - accuracy: 0.7404
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
727/804 [==========================>...] - ETA: 0s - loss: 0.5339 - accuracy: 0.7411
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5334 - accuracy: 0.7413

431/804 [===============>..............] - ETA: 0s - loss: 0.5290 - accuracy: 0.7466
Epoch 13: saving model to checkpoints/Optimization3\weights.13.hdf5
523/804 [==================>...........] - ETA: 0s - loss: 0.5304 - accuracy: 0.7453
Epoch 13: saving model to checkpoints/Optimization3\weights.13.hdf5
619/804 [======================>.......] - ETA: 0s - loss: 0.5308 - accuracy: 0.7434
Epoch 13: saving model to checkpoints/Optimization3\weights.13.hdf5
716/804 [=========================>....] - ETA: 0s - loss: 0.5320 - accuracy: 0.7431
Epoch 13: saving model to checkpoints/Optimization3\weights.13.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5333 - accuracy: 0.7421
Epoch 14/100
  1/804 [..............................] - ETA: 0s - loss: 0.4712 - accuracy: 0.8125
Epoch 14: saving model to checkpoints/Optimization3\weights.14.hdf5
101/804 [==>...........................] - ETA: 0s - loss: 0.5448 - accuracy: 0.7308
Epoch 14: saving model to checkpoints/Optimizati

496/804 [=================>............] - ETA: 0s - loss: 0.5330 - accuracy: 0.7434
Epoch 19: saving model to checkpoints/Optimization3\weights.19.hdf5
576/804 [====================>.........] - ETA: 0s - loss: 0.5331 - accuracy: 0.7425
Epoch 19: saving model to checkpoints/Optimization3\weights.19.hdf5
682/804 [========================>.....] - ETA: 0s - loss: 0.5340 - accuracy: 0.7406
Epoch 19: saving model to checkpoints/Optimization3\weights.19.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5336 - accuracy: 0.7416
Epoch 20/100
  1/804 [..............................] - ETA: 1s - loss: 0.5129 - accuracy: 0.7500
Epoch 20: saving model to checkpoints/Optimization3\weights.20.hdf5
 95/804 [==>...........................] - ETA: 0s - loss: 0.5195 - accuracy: 0.7553
Epoch 20: saving model to checkpoints/Optimization3\weights.20.hdf5
179/804 [=====>........................] - ETA: 0s - loss: 0.5278 - accuracy: 0.7458
Epoch 20: saving model to checkpoints/Optimizati

589/804 [====================>.........] - ETA: 0s - loss: 0.5359 - accuracy: 0.7392
Epoch 25: saving model to checkpoints/Optimization3\weights.25.hdf5
694/804 [========================>.....] - ETA: 0s - loss: 0.5341 - accuracy: 0.7403
Epoch 25: saving model to checkpoints/Optimization3\weights.25.hdf5
763/804 [===========================>..] - ETA: 0s - loss: 0.5330 - accuracy: 0.7421
Epoch 25: saving model to checkpoints/Optimization3\weights.25.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5330 - accuracy: 0.7420
Epoch 26/100
 86/804 [==>...........................] - ETA: 0s - loss: 0.5355 - accuracy: 0.7435
Epoch 26: saving model to checkpoints/Optimization3\weights.26.hdf5
168/804 [=====>........................] - ETA: 0s - loss: 0.5376 - accuracy: 0.7357
Epoch 26: saving model to checkpoints/Optimization3\weights.26.hdf5
274/804 [=========>....................] - ETA: 0s - loss: 0.5358 - accuracy: 0.7386
Epoch 26: saving model to checkpoints/Optimizati

...

386/804 [=============>................] - ETA: 0s - loss: 0.5269 - accuracy: 0.7473
Epoch 74: saving model to checkpoints/Optimization3\weights.74.hdf5
495/804 [=================>............] - ETA: 0s - loss: 0.5296 - accuracy: 0.7434
Epoch 74: saving model to checkpoints/Optimization3\weights.74.hdf5
592/804 [=====================>........] - ETA: 0s - loss: 0.5326 - accuracy: 0.7411
Epoch 74: saving model to checkpoints/Optimization3\weights.74.hdf5
681/804 [========================>.....] - ETA: 0s - loss: 0.5332 - accuracy: 0.7402
Epoch 74: saving model to checkpoints/Optimization3\weights.74.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5319 - accuracy: 0.7418
Epoch 75/100
  1/804 [..............................] - ETA: 2s - loss: 0.5174 - accuracy: 0.7188
Epoch 75: saving model to checkpoints/Optimization3\weights.75.hdf5
 91/804 [==>...........................] - ETA: 1s - loss: 0.5348 - accuracy: 0.7376
Epoch 75: saving model to checkpoints/Optimizati

465/804 [================>.............] - ETA: 0s - loss: 0.5340 - accuracy: 0.7400
Epoch 80: saving model to checkpoints/Optimization3\weights.80.hdf5
581/804 [====================>.........] - ETA: 0s - loss: 0.5336 - accuracy: 0.7396
Epoch 80: saving model to checkpoints/Optimization3\weights.80.hdf5
679/804 [========================>.....] - ETA: 0s - loss: 0.5331 - accuracy: 0.7409
Epoch 80: saving model to checkpoints/Optimization3\weights.80.hdf5
783/804 [============================>.] - ETA: 0s - loss: 0.5314 - accuracy: 0.7429
Epoch 80: saving model to checkpoints/Optimization3\weights.80.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5319 - accuracy: 0.7421
Epoch 81/100
 53/804 [>.............................] - ETA: 0s - loss: 0.5087 - accuracy: 0.7588
Epoch 81: saving model to checkpoints/Optimization3\weights.81.hdf5
174/804 [=====>........................] - ETA: 0s - loss: 0.5236 - accuracy: 0.7459
Epoch 81: saving model to checkpoints/Optimizati

549/804 [===================>..........] - ETA: 0s - loss: 0.5329 - accuracy: 0.7402
Epoch 86: saving model to checkpoints/Optimization3\weights.86.hdf5
644/804 [=======================>......] - ETA: 0s - loss: 0.5312 - accuracy: 0.7420
Epoch 86: saving model to checkpoints/Optimization3\weights.86.hdf5
751/804 [===========================>..] - ETA: 0s - loss: 0.5302 - accuracy: 0.7433
Epoch 86: saving model to checkpoints/Optimization3\weights.86.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5312 - accuracy: 0.7422
Epoch 87/100
 27/804 [>.............................] - ETA: 1s - loss: 0.5190 - accuracy: 0.7500
Epoch 87: saving model to checkpoints/Optimization3\weights.87.hdf5
150/804 [====>.........................] - ETA: 0s - loss: 0.5256 - accuracy: 0.7479
Epoch 87: saving model to checkpoints/Optimization3\weights.87.hdf5
247/804 [========>.....................] - ETA: 0s - loss: 0.5285 - accuracy: 0.7447
Epoch 87: saving model to checkpoints/Optimizati

610/804 [=====================>........] - ETA: 0s - loss: 0.5310 - accuracy: 0.7418
Epoch 92: saving model to checkpoints/Optimization3\weights.92.hdf5
724/804 [==========================>...] - ETA: 0s - loss: 0.5307 - accuracy: 0.7431
Epoch 92: saving model to checkpoints/Optimization3\weights.92.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5315 - accuracy: 0.7423
Epoch 93/100
  1/804 [..............................] - ETA: 1s - loss: 0.4606 - accuracy: 0.8125
Epoch 93: saving model to checkpoints/Optimization3\weights.93.hdf5
127/804 [===>..........................] - ETA: 1s - loss: 0.5304 - accuracy: 0.7409
Epoch 93: saving model to checkpoints/Optimization3\weights.93.hdf5
217/804 [=======>......................] - ETA: 1s - loss: 0.5309 - accuracy: 0.7399
Epoch 93: saving model to checkpoints/Optimization3\weights.93.hdf5
309/804 [==========>...................] - ETA: 0s - loss: 0.5296 - accuracy: 0.7416
Epoch 93: saving model to checkpoints/Optimizati

680/804 [========================>.....] - ETA: 0s - loss: 0.5306 - accuracy: 0.7430
Epoch 98: saving model to checkpoints/Optimization3\weights.98.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5317 - accuracy: 0.7420
Epoch 99/100
  1/804 [..............................] - ETA: 2s - loss: 0.4324 - accuracy: 0.8125
Epoch 99: saving model to checkpoints/Optimization3\weights.99.hdf5
 81/804 [==>...........................] - ETA: 1s - loss: 0.5319 - accuracy: 0.7407
Epoch 99: saving model to checkpoints/Optimization3\weights.99.hdf5
198/804 [======>.......................] - ETA: 1s - loss: 0.5279 - accuracy: 0.7438
Epoch 99: saving model to checkpoints/Optimization3\weights.99.hdf5
301/804 [==========>...................] - ETA: 0s - loss: 0.5304 - accuracy: 0.7425
Epoch 99: saving model to checkpoints/Optimization3\weights.99.hdf5
394/804 [=============>................] - ETA: 0s - loss: 0.5316 - accuracy: 0.7412
Epoch 99: saving model to checkpoints/Optimizati

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6024 - accuracy: 0.7259 - 369ms/epoch - 1ms/step
Loss: 0.6023790240287781, Accuracy: 0.7259474992752075


In [24]:
# Save to HDF5 file
nn.save('AlphabetSoupCharity_Optimization3.h5')